O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

In [64]:
import sklearn
sklearn.__version__

'1.6.1'

## Preparação de Dados

#### Importando bibliotecas

In [65]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from scipy.stats import randint

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

#### Baixando e analisando os dados

In [66]:
df = pd.read_csv('datasets/car_data.csv')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [68]:
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
299093,05/03/2016 17:44,0,sedan,1996,manual,90,golf,150000,0,gasoline,volkswagen,yes,05/03/2016 00:00,0,29525,07/04/2016 07:45
191933,02/04/2016 09:53,200,bus,2000,NaN,0,transporter,5000,0,gasoline,volkswagen,NaN,02/04/2016 00:00,0,78359,06/04/2016 07:44
36137,02/04/2016 15:47,8888,sedan,1990,auto,188,7er,150000,7,lpg,bmw,no,02/04/2016 00:00,0,20257,06/04/2016 14:44
14988,30/03/2016 13:38,1900,suv,1999,manual,98,other,125000,5,gasoline,daewoo,yes,30/03/2016 00:00,0,23843,07/04/2016 04:15
50594,27/03/2016 13:51,1000,sedan,2005,manual,88,c4,150000,5,petrol,citroen,yes,27/03/2016 00:00,0,55234,27/03/2016 13:51


In [69]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,354369.0,4416.656776,4514.158514,0.0,1050.0,2700.0,6400.0,20000.0
RegistrationYear,354369.0,2004.234448,90.227958,1000.0,1999.0,2003.0,2008.0,9999.0
Power,354369.0,110.094337,189.850405,0.0,69.0,105.0,143.0,20000.0
Mileage,354369.0,128211.172535,37905.341530,5000.0,125000.0,150000.0,150000.0,150000.0
RegistrationMonth,354369.0,5.714645,3.726421,0.0,3.0,6.0,9.0,12.0
NumberOfPictures,354369.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PostalCode,354369.0,50508.689087,25783.096248,1067.0,30165.0,49413.0,71083.0,99998.0


#### Convertendo tipos de dados

In [70]:
date_cols = ['DateCrawled', 'DateCreated', 'LastSeen']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce',  dayfirst=True)

#### Criando colunas úteis (ano e mês do anúncio, duração do anúncio em dias)

In [71]:
df['ListingYear'] = df['DateCreated'].dt.year
df['ListingMonth'] = df['DateCreated'].dt.month
df['AdDuration'] = (df['LastSeen'] - df['DateCreated']).dt.days

#### Removendo colunas inúteis para o modelo

In [72]:
df = df.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

#### Corrigindo valores absurdos

In [73]:
df = df[(df['RegistrationYear'] >= 1950) & (df['RegistrationYear'] <= 2025)]
df = df[(df['Power'] >= 40) & (df['Power'] <= 500)]
df = df[(df['Price'] >= 100) & (df['Price'] <= 200000)]

#### Preenchendo valores ausentes em categóricas com "unknown" e em numéricas com "0"

In [74]:
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('unknown')

#### Conferindo resultados do pré-processamento

In [75]:
df = df.reset_index(drop=True)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304192 entries, 0 to 304191
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              304192 non-null  int64 
 1   VehicleType        304192 non-null  object
 2   RegistrationYear   304192 non-null  int64 
 3   Gearbox            304192 non-null  object
 4   Power              304192 non-null  int64 
 5   Model              304192 non-null  object
 6   Mileage            304192 non-null  int64 
 7   RegistrationMonth  304192 non-null  int64 
 8   FuelType           304192 non-null  object
 9   Brand              304192 non-null  object
 10  NotRepaired        304192 non-null  object
 11  ListingYear        304192 non-null  int32 
 12  ListingMonth       304192 non-null  int32 
 13  AdDuration         304192 non-null  int64 
dtypes: int32(2), int64(6), object(6)
memory usage: 30.2+ MB


#### Conclusão da primeira etapa

Após preparar o DataFrame em diversas etapas de pré-processamento, agora temos um conjunto de dados consistente, com novas colunas úteis para o modelo, sem valores absurdos, sem colunas inúteis para a predição e sem valores NaN.

## Treinamento do modelo

#### Definindo target e features

In [77]:
target = df['Price']

features = df.drop('Price', axis=1)

In [78]:
cat_features = features.select_dtypes(include='object').columns
num_features = features.select_dtypes(exclude='object').columns

#### Dividindo treino e teste

In [79]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42)

#### Preparando dados para regressão linear

In [80]:
ohe_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)
    ],
    remainder='passthrough'
)

features_train_ohe = ohe_encoder.fit_transform(features_train)
features_test_ohe = ohe_encoder.transform(features_test)

scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train_ohe)
features_test_scaled = scaler.transform(features_test_ohe)

#### Preparando dados para árvore de decisão e floresta aleatória

In [81]:
features_train_enc = features_train.copy()
features_test_enc = features_test.copy()

for col in cat_features:
    encoder = LabelEncoder()
    combined = pd.concat([features_train_enc[col], features_test_enc[col]], axis=0)
    encoder.fit(combined)
    features_train_enc[col] = encoder.transform(features_train_enc[col])
    features_test_enc[col] = encoder.transform(features_test_enc[col])

#### Função para analisar e avaliar o modelo

In [82]:
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Modelo", cat_features=None):
    # tempo de treino
    start_train = time.time()
    if cat_features is not None:
        model.fit(X_train, y_train, cat_features=cat_features, verbose=False)
    else:
        model.fit(X_train, y_train)
    end_train = time.time()
    train_time = end_train - start_train

    # tempo de previsão
    start_pred = time.time()
    y_pred = model.predict(X_test)
    end_pred = time.time()
    pred_time = end_pred - start_pred

    # qualidade
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"{model_name} — RMSE: {rmse:.2f}, Treino: {train_time:.2f}s, Predição: {pred_time:.2f}s")
    
    return {
        'model': model_name,
        'rmse': rmse,
        'train_time': train_time,
        'pred_time': pred_time
    }

#### Treinando o modelo - Regressão linear

In [83]:
lr = LinearRegression()

#### Treinando o modelo - Árvore de decisão

In [84]:
dt = DecisionTreeRegressor(random_state=42, max_depth=10)

#### Treinando o modelo - Floresta aleatória

In [85]:
rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=20)

#### Treinando o modelo - Floresta aleatória (ajustado)

In [86]:
rf_tuned = RandomForestRegressor(
    random_state=42,
    n_estimators=200,
    max_depth=15,
    min_samples_leaf=2,
    max_features='sqrt',
    n_jobs=-1
)

#### Treinando o modelo - Gradient boosting com LightGBM

In [87]:
lgbm = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=-1,
    random_state=42
)

#### Treinando o modelo - Gradient boosting com LightGBM (ajustado)

In [88]:
lgbm_tuned = LGBMRegressor(
    random_state=42,
    n_estimators=400,
    learning_rate=0.05,
    max_depth=12,
    num_leaves=50,
    min_child_samples=10,
    subsample=0.8
)

#### Treinando o modelo - Gradient boosting com CatBoost

In [89]:
cat_cols = features.select_dtypes(include='object').columns.tolist()

cat_model = CatBoostRegressor(
    depth=8,
    learning_rate=0.1,
    n_estimators=300,
    loss_function='RMSE',
    random_state=42,
    verbose=False
)

In [90]:
print("NaNs em features_train:", np.isnan(features_train_scaled).sum())
print("NaNs em features_test:", np.isnan(features_test_scaled).sum())

NaNs em features_train: 0
NaNs em features_test: 0


#### Conclusão da segunda etapa

Separei os dados entre teste e treino e os preparei para cada modelo. Após isso, criei todos os modelos para comparação.

## Análise do modelo

#### Analisando o modelo - Regressão linear

In [91]:
results_lr = evaluate_model(lr, features_train_scaled, target_train, features_test_scaled, target_test, "Regressão Linear")

Regressão Linear — RMSE: 2524.04, Treino: 5.52s, Predição: 0.04s


#### Analisando o modelo - Árvore de decisão

In [92]:
results_dt = evaluate_model(dt, features_train_enc, target_train, features_test_enc, target_test, "Árvore de Decisão")

Árvore de Decisão — RMSE: 1958.83, Treino: 0.71s, Predição: 0.01s


#### Analisando o modelo - Floresta aleatória

In [93]:
results_rf = evaluate_model(rf, features_train_enc, target_train, features_test_enc, target_test, "Floresta Aleatória")

Floresta Aleatória — RMSE: 1545.86, Treino: 75.51s, Predição: 1.54s


#### Analisando o modelo - Floresta aleatória (ajustado)

In [94]:
results_rf_tuned = evaluate_model(
    rf_tuned,
    features_train_enc, target_train,
    features_test_enc, target_test,
    "Floresta Aleatória (Ajustado)"
)

Floresta Aleatória (Ajustado) — RMSE: 1649.70, Treino: 6.89s, Predição: 0.27s


#### Analisando o modelo - Gradient boosting com LightGBM

In [95]:
results_lgbm = evaluate_model(
    lgbm,
    features_train_enc, target_train,
    features_test_enc, target_test,
    "LightGBM (Gradient Boosting)"
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006659 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 700
[LightGBM] [Info] Number of data points in the train set: 243353, number of used features: 12
[LightGBM] [Info] Start training from score 4834.543223


c:\Users\joaod\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] O sistema não pode encontrar o arquivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\joaod\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\joaod\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\joaod\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^

LightGBM (Gradient Boosting) — RMSE: 1565.47, Treino: 1.56s, Predição: 0.12s


#### Analisando o modelo - Gradient boosting com LightGBM (ajustado)

In [96]:
results_lgbm_tuned = evaluate_model(
    lgbm_tuned,
    features_train_enc, target_train,
    features_test_enc, target_test,
    "LightGBM (Ajustado)"
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005743 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 703
[LightGBM] [Info] Number of data points in the train set: 243353, number of used features: 13
[LightGBM] [Info] Start training from score 4834.543223
LightGBM (Ajustado) — RMSE: 1559.07, Treino: 2.31s, Predição: 0.20s


#### Analisando o modelo - Gradient boosting com CatBoost

In [97]:
results_cat = evaluate_model(
    cat_model,
    features_train, target_train,
    features_test, target_test,
    model_name="CatBoost (Gradient Boosting)",
    cat_features=cat_cols
)

CatBoost (Gradient Boosting) — RMSE: 1557.72, Treino: 45.16s, Predição: 0.14s


#### Comparação entre modelos

In [98]:
results = pd.DataFrame([
    results_lr,
    results_dt,
    results_rf,
    results_rf_tuned,
    results_lgbm,
    results_lgbm_tuned,
    results_cat
])

display(results)

,model,rmse,train_time,pred_time
0,Regressão Linear,2524.039518,5.519799,0.042036
1,Árvore de Decisão,1958.829761,0.712353,0.010275
2,Floresta Aleatória,1545.863665,75.513091,1.539993
3,Floresta Aleatória (Ajustado),1649.699731,6.887735,0.274001
4,LightGBM (Gradient Boosting),1565.470807,1.555887,0.123986
5,LightGBM (Ajustado),1559.074517,2.310575,0.200080
6,CatBoost (Gradient Boosting),1557.715748,45.159499,0.144217


#### Interpretação dos modelos

- Regressão Linear

Serviu como baseline (modelo de referência).

RMSE mais alto (≈ 2548) indica que relações não lineares entre as variáveis são importantes para prever o preço.

Apesar de simples, foi mais lenta para treinar que as árvores — por causa da grande quantidade de features criadas pelo One-Hot Encoding.

- Árvore de Decisão

Reduziu o erro significativamente (de 2548 → 1968).

Treinou muito rápido e prevê quase instantaneamente.

Porém, é suscetível a overfitting, já que depende fortemente de divisões específicas dos dados.

- Floresta Aleatória

Trouxe melhor equilíbrio entre qualidade e estabilidade, reduzindo o REQM para ≈ 1562.

O tempo de treino aumentou bastante (muitas árvores), mas a precisão compensou.

Serviu como base de comparação direta com os métodos de boosting.

Após o ajuste de hiperparâmetros, a Floresta Aleatória ajustada apresentou RMSE ≈ 1662.9,
um valor ligeiramente maior, indicando que o novo conjunto de parâmetros priorizou generalização em vez de minimizar o erro absoluto.
Apesar de uma leve perda na métrica, o tempo de treinamento caiu de 70s para 17s,
mostrando uma melhor eficiência computacional e redução de sobreajuste.

- LightGBM (Gradient Boosting)

Obteve desempenho quase idêntico à Floresta Aleatória, com RMSE ≈ 1566.

Treinou muito mais rápido (apenas 5 segundos!) e previu com velocidade razoável.

Demonstra excelente eficiência computacional, sendo uma ótima escolha para uso em produção.

Com hiperparâmetros ajustados (menor taxa de aprendizado, mais iterações e limitação de profundidade),
o RMSE caiu ligeiramente para 1565.6, mantendo o tempo de treino dentro de 9s.
O modelo ajustado manteve alta precisão, com treinamento ainda eficiente,
confirmando que pequenas otimizações no aprendizado podem melhorar a estabilidade e reduzir o erro sem comprometer o desempenho.

- CatBoost (Gradient Boosting)

Teve o melhor RMSE geral (1561.7), empatando tecnicamente com a Floresta Aleatória, mas com tempo de inferência menor.

Consegue lidar diretamente com variáveis categóricas, dispensando o pré-processamento manual.

É o modelo mais equilibrado em termos de precisão, tempo de treino e simplicidade de uso.

#### Conclusão final

A regressão linear apresentou o maior REQM e serviu como baseline.
Modelos baseados em árvore reduziram o erro de forma expressiva.
O LightGBM foi o mais eficiente em tempo de treino, e o CatBoost obteve o menor erro total, sendo recomendado para o aplicativo.

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [x]  O código está livre de erros
- [x]  As células com o código foram organizadas em ordem de execução
- [x]  Os dados foram baixados e preparados
- [x]  Os modelos foram treinados
- [x]  A análise de velocidade e qualidade dos modelos foi realizada